# Iteration 3: Summary

Attempted to run for 50 epochs, batch size 32.

In [2]:
!pip install dill

In [ ]:
import dill
dill.load_session('notebook_env.db')

In [13]:
'''
import shutil
import os

source = './food-101/test/ramen'
dest = './food-101/valid/ramen'

files = os.listdir(source)

i = 0
for f in files:
    if i < 175:
        file= os.path.join(source, f)
        shutil.move(file,dest)
        print("adding img ", i)
    i+=1
'''

'\nimport shutil\nimport os\n\nsource = \'./food-101/test/ramen\'\ndest = \'./food-101/valid/ramen\'\n\nfiles = os.listdir(source)\n\ni = 0\nfor f in files:\n    if i < 175:\n        file= os.path.join(source, f)\n        shutil.move(file,dest)\n        print("adding img ", i)\n    i+=1\n'

# Preprocessing and Loading Data

For loading the data, I first used the train/test split that was provided with our dataset.  This gave us 750 images in train and 250 in test PER class.  Then, I split the test set into test & validation.  I moved 175 images to the validation set, thus we are left with 75 images in test.

For image preprocessing, I first resize each image to our IMG_SIZE constant, currently set to 224 pixels.  I then recolor the image to RGB.  I also assign the labels to the images and shuffle the dataset.


After loading the data, the shape of X (features) will be (-1 {this means any number of features}, IMG_SIZE, IMG_SIZE, 3 {number of channels - RGB}), and the shape of y (labels) will be (-1, 1).

In [3]:
# importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

import cv2
import random
import datetime
import time
import shutil
import stat

import collections
from collections import defaultdict

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical, plot_model

from keras.models import Model

Using TensorFlow backend.


In [4]:
# constants
IMG_SIZE = 224
NUM_CLASSES = 5
noodle_classes = ['pad_thai','pho','ramen','spaghetti_bolognese','spaghetti_carbonara']

# these are the main variables we tested and documented
EPOCHS = 50
BATCH_SZ = 32
LEARN_RATE = 0.0001

In [6]:
# function to create the datasets (train, test, and valid)
# preprocesses the images (resize/recolor), assigns labels, and shuffles the dataset
root_dir = 'food-101/images/'

def create_data(root, _name):
    data_set = []

    for noodle_class in noodle_classes:
        path = os.path.join(root, noodle_class)
        label = noodle_classes.index(noodle_class) # assign an integer label to the image based on our noodle_classes array
        print(noodle_class,label)

        for img in os.listdir(path): # iterates through each image in the noodle folder
                if img.startswith('.'):
                    continue
                # each image is a 2D array of RGB value
                try:
                    img_arr = cv2.imread(os.path.join(path,img)) 
                    img_to_rgb = img_arr[:,:,::-1] # recolor
                    new_img = cv2.resize(img_to_rgb,(IMG_SIZE,IMG_SIZE)) #resize
                    data_set.append([new_img,label]) # store image and label together in dataset so we can shuffle without images getting mislabeled
                except Exception as e: # catch bad images
                    print("create exception: ",e)
        
    # randomize
    random.shuffle(data_set)
    
    X = []
    y = []

    for features, label in data_set:
        X.append(features) # 2D array of RGB values representing features
        y.append(label) # integer representing class/label

    # reshape X and y
    X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    y = np.array(y).reshape(-1, 1)
    print('X', X.shape)
    print('y', y.shape)

    return X,y

In [8]:
# load data in train, test, and valid

X_train, y_train = create_data('food-101/train', "train")
X_test, y_test = create_data('food-101/test', "test")
X_valid, y_valid = create_data('food-101/valid', "valid")

pad_thai 0


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'food-101/train\\pad_thai'

# Data Normalization and Augmentation
Here I am normalizing the data to scale our input training vectors. This will help improve accuracy and increase training speed.

Currently, I am attempting to implement image augmentation to improve accuracy as well.

In [ ]:
# normalize data
# X_train = X_train/255.0
# X_test = X_test/255.0

#print('X train before normalize',X_train[1])
print('y train before normal',y_train)
# input('wait1')

# L2-normalizes the given array, i.e., it makes the sum of squares of each element of the array to be equal to one
X_train = tf.keras.utils.normalize(X_train, axis=1)
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)

#print('X train after normal',X_train[1])
print('y train after normal',y_train)

X_test = tf.keras.utils.normalize(X_test, axis=1)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

X_valid = tf.keras.utils.normalize(X_valid, axis=1)
y_valid = tf.keras.utils.to_categorical(y_valid, NUM_CLASSES)

In [ ]:
# image augmentation - for better performance
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True)

datagen.fit(X_train)

# VGGNet 16
This is the 16-layer VGGNet architecture which consists of twelve convolutional layers, some followed by max pooling layers, four fully-connected layers, and lastly a softmax classifier.

The "VGG-blocks" is what stood out about this paper.  These blocks contain the convolutional layers whcich use small 3x3 filters followed by a max pooling layer with size 2x2 and stride 2.

See below for a chart depicting the architecture.

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model

# creating the actual model using the VGGNet16 architecture

input_shape = (IMG_SIZE, IMG_SIZE, 3) # 224x224x3 RGB image

#Instantiate an empty model
model = Sequential([
Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'), # first two convolutional layers
Conv2D(64, (3, 3), activation='relu', padding='same'), # 64 filters | 224x224x64
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), # image dim -> 112x112x64
Conv2D(128, (3, 3), activation='relu', padding='same'), # 128 filters, size 3x3, stride 1
Conv2D(128, (3, 3), activation='relu', padding='same',), 
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), # image dim -> 56x56x128
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
Conv2D(256, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), # 28x28x256
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), # 14x14x512
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
Conv2D(512, (3, 3), activation='relu', padding='same',),
MaxPooling2D(pool_size=(2, 2), strides=(2, 2)), # 7x7x512
Flatten(), # flatten through fully connected layer with 25088 filters size 1x1
Dense(4096, activation='relu'), # 2 more fully connected layers
Dense(4096, activation='relu'),
Dense(5, activation='softmax') # softmax output layer, 5 possible values/classes
])

model.summary()

plot_model(model, show_shapes=True, to_file='vgg_block.png')

# Training

In [ ]:
# compiling model and training

opt = optimizers.SGD(lr=LEARN_RATE, momentum=.9) # momentum help accelerate gradient vectors in the right directions

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=['accuracy'])
'''


# Compile the model
model.compile(loss="categorical_crossentropy", optimizer='SGD', metrics=['accuracy'])
'''
print(X_train.shape)
print(y_train.shape)

'''
# fitting model without image augmentation
model.fit(X_train, 
          y_train, 
          batch_size=BATCH_SZ, 
          epochs=EPOCHS,     
          validation_data=(X_valid, y_valid))
'''

# fitting model with image augmentation
model.fit_generator(datagen.flow(X_train, y_train, batch_size=BATCH_SZ),
                    steps_per_epoch=len(X_train) / 64,
                    epochs=EPOCHS,
                    validation_data=(X_valid, y_valid))

# Evaluation

In [ ]:
# evaluate model on test data
scores = model.evaluate(X_test, y_test, verbose=1)
# verbose indiciator to display training progress info
# 1 (true) (default) | 0 (false)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [6]:
# Keeping track of overall results
import pandas as pd

print("Before Image Augmentation:")
data = [[30, 32, 0.0001, 1.6084, 0.4133],
        [40, 64, 0.0001, 1.6089, 0.2987]]
pd.DataFrame(data, columns=["Epochs", "Batch Size", "Learning Rate", "Test Loss", "Test Accuracy"])

Before Image Augmentation:


,Epochs,Batch Size,Learning Rate,Test Loss,Test Accuracy
0,30,32,0.0001,1.6084,0.4133
1,40,64,0.0001,1.6089,0.2987
